In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
import torch,os
torch.set_printoptions(edgeitems=2, threshold=50)

drive.mount('/content/gdrive/', force_remount=True)


import pathlib
data_dir = '/content/gdrive/MyDrive/bike-sharing-dataset'

imgdir_path = pathlib.Path('/content/gdrive/MyDrive/bike-sharing-dataset')

file_list = sorted([str(path) for path in imgdir_path.glob('*.csv')])

print(file_list)

Mounted at /content/gdrive/
['/content/gdrive/MyDrive/bike-sharing-dataset/day.csv', '/content/gdrive/MyDrive/bike-sharing-dataset/hour-fixed.csv', '/content/gdrive/MyDrive/bike-sharing-dataset/hour.csv']


In [8]:
#https://likegeeks.com/numpy-loadtxt-tutorial/
bikes_numpy = np.loadtxt(
    "/content/drive/MyDrive/bike-sharing-dataset/hour-fixed.csv", 
    dtype=np.float32, 
    delimiter=",", 
    skiprows=1, 
    converters={1: lambda x: float(x[8:10])}) # <1>
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

LEARNNG

In [6]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
second_point = points[1]
second_point.storage_offset()

2

The stride is a tuple indicating the number of elements in the storage that have to be skipped when the index is increased by 1 in each dimension. For instance, our points tensor has a stride of (2, 1):

In [7]:
points.stride()

(2, 1)

In [4]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

Now let’s reshape the data to have 3 axes--day, hour, and then our 17 columns:

In [5]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

But the real crux of this code is the call to view, which is really important: it changes the way the tensor looks at the same data as contained in storage.

As you learned in the previous chapter, calling view on a tensor returns a new tensor that changes the number of dimensions and the striding information, without changing the storage. This means we can rearrange our tensor at basically zero cost, because no data will be copied. Our call to view requires us to provide the new shape for the returned tensor. We use -1 as a placeholder for “however many indexes are left, given the other dimensions and the original number of elements.”

In [9]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [10]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [11]:
weather_onehot.scatter_(
    dim=1, 
    index=first_day[:,9].unsqueeze(1).long() - 1, # <1>
    value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [12]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])